In [10]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta



In [2]:
tweets = pd.read_csv("tweets.csv")
tweetI = pd.read_csv("tweet-index.csv")
df = pd.read_csv("AMZN.csv")

In [3]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-01-03,37.896000,37.938000,37.384998,37.683498,37.683498,70422000
1,2017-01-04,37.919498,37.984001,37.709999,37.859001,37.859001,50210000
2,2017-01-05,38.077499,39.119999,38.013000,39.022499,39.022499,116602000
3,2017-01-06,39.118000,39.972000,38.924000,39.799500,39.799500,119724000
4,2017-01-09,39.900002,40.088501,39.588501,39.846001,39.846001,68922000


In [4]:
tweets.post_date = tweets.post_date.astype(str)
i = 0
for tweet in tweets.post_date:
    tweets.loc[i, 'post_date'] = datetime.utcfromtimestamp(int(tweets.loc[i, 'post_date'])).strftime('%Y-%m-%d')
    i = i+1

KeyboardInterrupt: 

In [ ]:
merged_df = pd.merge(tweets, tweetI, how='inner', on='tweet_id')

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('merged_file.csv', index=False)

In [12]:
mergedFile = pd.read_csv("merged_file.csv")

In [7]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

/Users/ashutoshsingh/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
def flatten(lst):
    """
    Flattens a 2D list into a 1D list.
    """
    flattened = []
    for sublist in lst:
        for item in sublist:
            flattened.append(item)
    return flattened

In [14]:
dates = ["2018-08-30", "2018-08-15","2018-07-30","2018-07-15","2018-06-30","2018-06-15","2018-05-30","2018-05-15","2018-04-30","2018-04-15",
"2017-08-30", "2017-08-15","2017-07-30","2017-07-15","2017-06-30","2017-06-15","2017-05-30","2017-05-15","2017-04-30","2017-04-15"]
x_train = []
y_train = []
for targetDate in dates:
    target_date = pd.to_datetime(targetDate)
    one_week_before = target_date - timedelta(days=7)
    one_week_before = one_week_before.strftime('%Y-%m-%d')
    start_value = one_week_before
    end_value = targetDate
    # Use boolean indexing to select tuples that fall within the specified range
    resultPart = mergedFile[(mergedFile['post_date'] >= start_value) & (mergedFile['post_date'] <= end_value)]
    resultCompany = resultPart[resultPart['ticker_symbol'] == "AMZN"]
    resultCompany = resultCompany.sort_values('retweet_num', ascending=False)
    resultCompany = resultCompany.head(200)
    # tweetsOb = result['body']
    tweetsOb = resultCompany['body'].values
    retweetsOb = resultCompany['retweet_num'].values
    commentOb = resultCompany['comment_num'].values
    likeOb = resultCompany['like_num'].values
    
    daysOb = [(datetime.strptime(targetDate, "%Y-%m-%d")-datetime.strptime(date2, "%Y-%m-%d")).days for date2 in resultCompany['post_date']]
    sentimentResult = sentiment_pipeline(tweetsOb.tolist())
    labels = [int(i['label'].split()[0]) for i in sentimentResult]
    xComp = [[a,b,c,d,e] for a,b,c,d,e in zip(labels, retweetsOb, commentOb, likeOb, daysOb)]    
#     xComp = flatten(xComp)

    stockTargetDate = target_date + timedelta(days=1)
    six_days_before = stockTargetDate - timedelta(days=6)
    six_days_before = six_days_before.strftime('%Y-%m-%d')

    start_value = six_days_before
    end_value = stockTargetDate.strftime('%Y-%m-%d')

    # Use boolean indexing to select tuples that fall within the specified range
    st = df[(df['Date'] >= start_value) & (df['Date'] <= end_value)]
    diff = st.Close.tolist()[-1] - st.Open.tolist()[0]
    delta = diff / st.Open.tolist()[0]
    delta
    for elem in xComp:
        x_train.append(elem)
        y_train.append(delta)

In [15]:
np.array(x_train).shape

(4000, 5)

In [18]:
np.array(y_train)[:5]

array([0.05102348, 0.05102348, 0.05102348, 0.05102348, 0.05102348])

In [36]:
arr = np.array(x_train)

ayy = np.array(y_train).shape

xDf = pd.DataFrame(arr, columns=['A', 'B', 'C','D','E'])
yDf = pd.DataFrame(y_train, columns=['values'])

# Export the DataFrame to a CSV file
xDf.to_csv('outputx.csv', index=False)
yDf.to_csv('outputy.csv', index=False)

In [35]:
yDf

,values
0,0.051023
1,0.051023
2,0.051023
3,0.051023
4,0.051023
...,...
3995,-0.016629
3996,-0.016629
3997,-0.016629
3998,-0.016629


In [33]:
my_array = [1, 2, 3, 4, 5]

# Convert array to dataframe
newdf = pd.DataFrame(my_array, columns=['Values'])

# Display dataframe
print(newdf)

   Values
0       1
1       2
2       3
3       4
4       5
